python 提供 with 独特语法，可让程序设计者更容易管理这些开放资源。这样的语法架构之下，python 程序会自动进行资源的建立、清理和回收，使得使用资源时更为方便。

传统上若要打开一个档案，我们会：

```
#开启档案
f = open(filename)

#...

#关闭档案
```

这种写法的问题：如果过程中发生意外，程序提早跳开，这个开启的档案没法被关闭，所以比较好的写法是使用 try ... finally

```
#开启档案
f = open(filename)

try : 
    # ...

finally:
    #关闭档案
    f.close()
```


这种写法虽然不会有问题，但是必须要手动关闭档案的程序，不方便所以改成 with 的写法：

```
with open(filename) as f:
    #...
```

这里在使用 with 开启档案时候，会将开启的档案放在 f 变数中，但是这个 f 只会在这个 wtih 的范围内可以使用，而离开这个范围 f 就会自动关闭，回收资源。

实际上，在with后面的代码块抛出异常时，exit()方法被执行。开发库时，清理资源，关闭文件等操作，都可以放在exit()方法中。 

例子：

```
#以 with 开启并写入档案
with open('file.txt', 'w') as f:
    f.write('Hello, world!')
```

例子 2：
```
try:
    fileReader = open('students.txt', 'r')

    for row in fileReader:
        print(row.strip())
except:
    print('Read file failed')
finally:
    fileReader.close()
```
写成 with 形式：

```
#!/usr/bin/env python

with open('students.txt', 'r') as fileReader:
    for row in fileReader:
        print(row.strip())
```

如果有多项，可以这样写：

```
With open('1.txt') as f1, open('2.txt') as  f2:
    do something
```


### 自行建立 Context Manager

若要自行建立 Context Manager, 只要定义好类别的__enter__与__exit__两个方法函数。with 在刚开始执行时，会执行__enter__，传回配给资源，在 with 范围结束时，会自动呼叫__exit__释放资源（例如关闭档案）。

一个打开档案的 context manager 例子:
```
#自行定义 context Manager
class File(object):
    def __init__(self, filename, mode):
        #设定档案名与开档模式
        self.filename = filename
        self.mode = mode
    
    #配给资源（开启档案）
    def __enter__(self):
        print('开启档案：' + self.filename)
        self.open_file = open(self.filename, self.mode)
        return self.open_file
    
    #回收资源（关闭档案）
    def __exit__(self, type, value, traceback):
        print('关闭档案：' + self.filename)
        self.open_file.close()
```

使用方法如下：

```
with File('file.txt', 'w') as f:
    print('写入档案... ')
    f.write('Hello, world.')
```

自定义上下文管理器模板 2：

```
class DBManager(object):
    def __init__(self):
        pass

    def __enter__(self):
        print('__enter__')
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__')
        return True

def getInstance():
        return DBManager()

with getInstance() as dbManagerIns:
    print('with demo')
```

代码运行结果如下：
```
__enter__
with demo
__exit__
```

### contextlib 的模组的 decorator （装饰器实现 context manager)

例：

```
from contextlib import contextmanager

#自行定义 context manager
@contextmanager
def open_file(name, mode):
    #配给资源（开启档案）
    f = open(name, mode)
    yield f
    #回收资源（关闭档案）
    f.close()
```

使用方式：

```
with open_file('file.txt', 'w') as f:
    f.write('Hello, world.')
```

### 原理：
深挖一下with语句的原理
要搞清楚with语句的原理，先要说一下下面这几个概念：

上下文管理协议（Context Management Protocol）：包含方法 __enter__()和__exit__()，支持该协议的对象要实现这两个方法。
上下文管理器（Context Manager）：支持上下文管理协议的对象，这种对象实现了__enter__()和__exit__()方法。上下文管理器定义执行with语句时要建立的运行时上下文，负责执行with语句块上下文中的进入与退出操作。通常使用with语句调用上下文管理器，也可以通过直接调用其方法来使用。
说完上面两个概念，我们再从with语句的常用表达式入手，一段基本的with表达式，其结构是这样的：

```
with EXPR as VAR:
    BLOCK
```

其中EXPR可以是任意表达式；as VAR是可选的。其一般的执行过程是这样的：

1. 执行EXPR，生成上下文管理器context_manager；
2. 获取上下文管理器的__exit()__方法，并保存起来用于之后的调用；
3. 调用上下文管理器的__enter__()方法；如果使用了as子句，则将__enter__()方法的返回值赋值给as子句中的VAR；
4. 执行BLOCK中的表达式；
5. 不管是否执行过程中是否发生了异常，执行上下文管理器的__exit__()方法，__exit__()方法负责执行“清理”工作，如释放资源等。如果执行过程中没有出现异常，或者语句体中执行了语句break/continue/return，则以None作为参数调用__exit__(None, None, None)；如果执行过程中出现异常，则使用sys.exc_info得到的异常信息为参数调用__exit__(exc_type, exc_value, exc_traceback)；
6. 出现异常时，如果__exit__(type, value, traceback)返回False，则会重新抛出异常，让with之外的语句逻辑来处理异常，这也是通用做法；如果返回True，则忽略异常，不再对异常进行处理。